In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
#import plotly.express as px

listings1 = pd.read_csv("C:/Users/marioreyes/Desktop/Data Analysis Practice/Python/Project/airbnb_part1.csv")
listings2 = pd.read_csv("C:/Users/marioreyes/Desktop/Data Analysis Practice/Python/Project/airbnb_part2.csv")
listings =  pd.concat([listings1, listings2])
listings.head()

print(listings.columns)
print(listings1)
print(listings2)
print(listings)

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [ ]:
#Keeping columns we want to investigate
listings = listings[['id', 'name', 'neighborhood_overview', 'host_id',
        'host_name', 'host_since',
        'host_response_rate', 'host_acceptance_rate',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'room_type', 'accommodates', 'bathrooms',
        'bedrooms', 'beds', 'price', 'number_of_reviews', 
        'review_scores_rating', 'review_scores_accuracy', 'availability_365',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location','reviews_per_month']]
#checking that the number of rows is the sum of listings1 and listings2
print(len(listings))

37548


In [ ]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37548 entries, 0 to 16917
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            37548 non-null  float64
 1   name                          37546 non-null  object 
 2   neighborhood_overview         20574 non-null  object 
 3   host_id                       37548 non-null  int64  
 4   host_name                     37543 non-null  object 
 5   host_since                    37543 non-null  object 
 6   host_response_rate            22547 non-null  object 
 7   host_acceptance_rate          22565 non-null  object 
 8   neighbourhood_cleansed        37548 non-null  object 
 9   neighbourhood_group_cleansed  37548 non-null  object 
 10  latitude                      37548 non-null  float64
 11  longitude                     37548 non-null  float64
 12  room_type                     37548 non-null  object 
 13  accomm

In [ ]:
#Coverting columsn to correct data types
listings['price'] = listings['price'].replace('[^\\d.]','',regex=True).astype(float)
listings['host_response_rate']=listings['host_response_rate'].astype(str).str.strip('%').astype(float)/100
listings['host_acceptance_rate']=listings['host_acceptance_rate'].astype(str).str.strip('%').astype(float)/100
print(listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 37548 entries, 0 to 16917
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            37548 non-null  float64
 1   name                          37546 non-null  object 
 2   neighborhood_overview         20574 non-null  object 
 3   host_id                       37548 non-null  int64  
 4   host_name                     37543 non-null  object 
 5   host_since                    37543 non-null  object 
 6   host_response_rate            22547 non-null  float64
 7   host_acceptance_rate          22565 non-null  float64
 8   neighbourhood_cleansed        37548 non-null  object 
 9   neighbourhood_group_cleansed  37548 non-null  object 
 10  latitude                      37548 non-null  float64
 11  longitude                     37548 non-null  float64
 12  room_type                     37548 non-null  object 
 13  accomm

In [ ]:
#Dropping rows with NA prices
listings = listings.dropna(subset = ['price'], axis=0)

#Average price of rentals by hosts.
listings.groupby('host_id')['price'].mean().sort_values(ascending=False).to_frame()
#Getting the top 5 hosts with the largest average price
top_5 = listings.groupby('host_id')['price'].mean().sort_values(ascending=False).to_frame().index[0:5]

#Investigating the rentals of the highest price
listings[listings['host_id'].isin(top_5)]


#looks like many of these listings have N/A values for locations. 
#We will get rid of any rows with null values in location
listings = listings.dropna(subset = ['neighbourhood_cleansed'], axis=0)
listings = listings.dropna(subset = ['neighbourhood_group_cleansed'], axis=0)

#Removing any duplicates
listings.duplicated().sum()
listings.drop_duplicates(inplace=True)

In [ ]:
#Checking the number of null values in each column
print(listings.isna().sum())

#Looks like there are some reviews/reponses/house info that are N/A. For this practice we will treat N/A as 0
listings.fillna(0, inplace=True)
listings.isna().sum()


id                                  0
name                                0
neighborhood_overview           10266
host_id                             0
host_name                           0
host_since                          0
host_response_rate               3257
host_acceptance_rate             3517
neighbourhood_cleansed              0
neighbourhood_group_cleansed        0
latitude                            0
longitude                           0
room_type                           0
accommodates                        0
bathrooms                           5
bedrooms                           53
beds                              148
price                               0
number_of_reviews                   0
review_scores_rating             6703
review_scores_accuracy           6703
availability_365                    0
review_scores_cleanliness        6703
review_scores_checkin            6703
review_scores_communication      6703
review_scores_location           6704
reviews_per_

In [ ]:
#Data Analysis
#What is the number of listing per neighborhood
listings.groupby('neighbourhood_cleansed')['neighbourhood_cleansed'].count().sort_values(ascending=False)


#Number of listing for the top 20 neighborhoods
#Creating new dataframe with only those neighborhooods
top20=listings.groupby('neighbourhood_cleansed')['neighbourhood_cleansed'].count().sort_values(ascending=False).index[0:20]
top20_neighborhoods = listings[listings['neighbourhood_cleansed'].isin(top20)]

#To sort the histograph we need to sort the dataframe. First we count the number of values in each category
counts = top20_neighborhoods['neighbourhood_cleansed'].value_counts()
#Sort the DataFrame based on the counts
top20_neighborhoods_sorted = top20_neighborhoods.sort_values(by='neighbourhood_cleansed', key=lambda x: counts[x], ascending=False)

#Visualizing new sorted top 20 neighborhoods. We are adding counts of neighborhood and median price in same graph
fig, ax1 = plt.subplots()
plt.xticks(rotation=45, horizontalalignment='right') #Adding xtick to rotate xlabels. Only works if we add before ax2=ax1.twinx()

ax2 = ax1.twinx() # secondary y-axis
ax3 = sns.histplot( # bar chart
    data=top20_neighborhoods_sorted,
    x='neighbourhood_cleansed',
    ax=ax1 #Making this our 1st y axis
)

ax4 = sns.lineplot(data=top20_neighborhoods_sorted, #line chart
                   x="neighbourhood_cleansed",  
                   y="price", 
                   color='orange', 
                   estimator='median', 
                   ax=ax2, #making this our 2nd y axis
                   errorbar=None #gets rid of the line chart intervals
                  )

ax3.bar_label(ax3.containers[0],fontsize=.5 ) #adding labels to bar chart
plt.show()

In [ ]:
#Adding single line chart to compare to previous visual. The visuals look accurate
ax2 = sns.lineplot(data=top20_neighborhoods_sorted, x="neighbourhood_cleansed",  y="price", color='orange', estimator='median', errorbar=None)
plt.xticks(rotation=45, horizontalalignment='right')
plt.show()

top20_neighborhoods_sorted.groupby('neighbourhood_cleansed')['price'].median().sort_values(ascending=False)

In [ ]:
#Data Analysis Conclusion.
#Bedford-Stuyvesant has the largest number of airbnb listings but they rank 15 in terms of highest median price. The lower price compared to the other neighborhoods explains why it has high amounts of listings
#This explains why midtown, which has the lowest number of listings has a the highest median price
#Yet the second largest number of airbnb listings are in Midtown, which has ranks 3rd in highest price. 

In [ ]:
#Data Analysis 2
#Confirming correlation between price and neighborhood
correlation_matrix = listings.corr(numeric_only=True, method="pearson")
plt.figure(figsize=(11,11), dpi = 50)

sns.heatmap(correlation_matrix, annot=True)
plt.title("Correlation Metrix for Numeric Features")

plt.yticks(rotation=0)
plt.show()

In [ ]:
#The column "neighborhood" is not numeric. To check correlation we will first convert non numeric columns to numbers

In [ ]:
listings['neighbourhood_group_cleansed']

0        Manhattan
1         Brooklyn
2        Manhattan
3        Manhattan
5         Brooklyn
           ...    
16913     Brooklyn
16914     Brooklyn
16915     Brooklyn
16916        Bronx
16917        Bronx
Name: neighbourhood_group_cleansed, Length: 22719, dtype: object


In [ ]:
#converting object columns to numbers
df_numerized = listings.copy()

for col_name in df_numerized.columns:
    if(df_numerized[col_name].dtype == "object"):
        df_numerized[col_name] = df_numerized[col_name].astype("category") 
        df_numerized[col_name] = df_numerized[col_name].cat.codes

print(df_numerized)

                 id   name  ...  review_scores_location  reviews_per_month
0      2.595000e+03  17681  ...                    4.81               0.27
1      6.848000e+03  14860  ...                    4.69               1.04
2      6.872000e+03  20993  ...                    5.00               0.03
3      6.990000e+03  20764  ...                    4.85               1.37
5      7.097000e+03  15183  ...                    4.95               2.16
...             ...    ...  ...                     ...                ...
16913  1.280000e+18   9283  ...                    0.00               0.00
16914  1.280000e+18  12657  ...                    0.00               0.00
16915  1.280000e+18  13635  ...                    0.00               0.00
16916  1.280000e+18   2812  ...                    0.00               0.00
16917  1.280000e+18    451  ...                    0.00               0.00

[22719 rows x 27 columns]


In [ ]:
#Plotting all numeric columns in a correlation matrix
correlation_matrix = df_numerized.corr(numeric_only=True, method="pearson")
plt.figure(figsize=(14,14), dpi = 40)
sns.heatmap(correlation_matrix, annot=True)

plt.title("Correlation Metrix for Numeric Features")

plt.yticks(rotation=0)
plt.show()

In [ ]:
listings

                 id  ... reviews_per_month
0      2.595000e+03  ...              0.27
1      6.848000e+03  ...              1.04
2      6.872000e+03  ...              0.03
3      6.990000e+03  ...              1.37
5      7.097000e+03  ...              2.16
...             ...  ...               ...
16913  1.280000e+18  ...              0.00
16914  1.280000e+18  ...              0.00
16915  1.280000e+18  ...              0.00
16916  1.280000e+18  ...              0.00
16917  1.280000e+18  ...              0.00

[22719 rows x 27 columns]


In [ ]:
#Looking at the matrix, it turns out the price and "neighborhood_cleansed" does not have a high correlation.
#The matrix does give us other columns that have high correlation with each other


In [ ]:
#The matrix is a bit too big to spot all of the high correlation. Let's convert it to a table
correlation_mat = df_numerized.corr(numeric_only=True)

corr_pairs = correlation_mat.unstack()
print(corr_pairs)

id                 id                             1.000000
                   name                           0.009762
                   neighborhood_overview         -0.194844
                   host_id                        0.427655
                   host_name                      0.028509
                                                    ...   
reviews_per_month  review_scores_cleanliness      0.290912
                   review_scores_checkin          0.284610
                   review_scores_communication    0.286429
                   review_scores_location         0.283211
                   reviews_per_month              1.000000
Length: 729, dtype: float64


In [ ]:
#Sorting values by highest to lowest
sorted_pairs = corr_pairs.sort_values(ascending=False)

print(sorted_pairs)

reviews_per_month            reviews_per_month              1.000000
id                           id                             1.000000
review_scores_checkin        review_scores_checkin          1.000000
neighborhood_overview        neighborhood_overview          1.000000
review_scores_communication  review_scores_communication    1.000000
                                                              ...   
review_scores_location       id                            -0.453372
id                           review_scores_checkin         -0.455553
review_scores_checkin        id                            -0.455553
review_scores_accuracy       id                            -0.455747
id                           review_scores_accuracy        -0.455747
Length: 729, dtype: float64


In [ ]:
#Only showing pairs with high correlation
sorted_pairs[((sorted_pairs) >0.5) & (sorted_pairs<1)]

review_scores_rating         review_scores_accuracy         0.995737
review_scores_accuracy       review_scores_rating           0.995737
review_scores_communication  review_scores_rating           0.994137
review_scores_rating         review_scores_communication    0.994137
review_scores_checkin        review_scores_communication    0.993344
review_scores_communication  review_scores_checkin          0.993344
review_scores_rating         review_scores_cleanliness      0.993164
review_scores_cleanliness    review_scores_rating           0.993164
review_scores_accuracy       review_scores_communication    0.993152
review_scores_communication  review_scores_accuracy         0.993152
review_scores_checkin        review_scores_rating           0.992323
review_scores_rating         review_scores_checkin          0.992323
review_scores_cleanliness    review_scores_accuracy         0.991786
review_scores_accuracy       review_scores_cleanliness      0.991786
review_scores_checkin        revie

In [ ]:
#Data Analysis 2 Conclusion
#looks like there's a high correlation between review scores.

In [ ]:
listings['neighbourhood_group_cleansed']

0        Manhattan
1         Brooklyn
2        Manhattan
3        Manhattan
5         Brooklyn
           ...    
16913     Brooklyn
16914     Brooklyn
16915     Brooklyn
16916        Bronx
16917        Bronx
Name: neighbourhood_group_cleansed, Length: 22719, dtype: object


In [ ]:
#Data Analysis 3
#Let's see the availability of neighborhoods throughout the whole year

box = sns.boxplot(data=listings, 
                  x='neighbourhood_group_cleansed',
                  y='availability_365', 
                  hue='neighbourhood_group_cleansed',
                  legend=True,
                  showmeans=True, #Optional. You can show the mean in the graph
                  meanprops={'marker':'o','markerfacecolor':'white','markeredgecolor':'blue','markersize':'5'} #Desing changes for mean
                  )

#This line will show dots for each listing
#sns.stripplot(x="neighbourhood_group_cleansed", y="availability_365", data=listings, color="black", size=4, ax=box)

#You can increase the box legend axis
box.legend(prop=dict(size=10))

#Can change axis font values
sns.set_theme(font_scale=1)


plt.show()

In [ ]:
#Data Analysis Conclusion
#Bronx listings had the most availability throughout the year. Listings in Manhattan had the lowest number of avaiablity throughout the year

In [ ]:
##Data Analysis 5
#Let's see how many room types there are in NYC

#print(listings['room_type'])
print(listings.groupby(['room_type'])['room_type'].count().sort_values(ascending=False))


print(listings.groupby(['room_type'])['room_type'].count().sort_values(ascending=False)/listings['room_type'].count()*100)
ax5 = sns.countplot(
    data=listings,
    x='neighbourhood_group_cleansed',
    hue='room_type'
)


#Shows the value of the first container
#ax5.bar_label(ax5.containers[0])

#Shows value in all containers and customize fontsize
for container in ax5.containers:
    ax5.bar_label(container,fontsize=9)


plt.show()

room_type
Entire home/apt    12536
Private room        9583
Hotel room           354
Shared room          246
Name: room_type, dtype: int64
room_type
Entire home/apt    55.178485
Private room       42.180554
Hotel room          1.558167
Shared room         1.082794
Name: room_type, dtype: float64


In [ ]:
#Data Analysis 5
#Most of the room types in NYC are entire homes or apartments.
#Although most of these room types are entire homes/apartments, most of these reside in Manhatann. Brooklyn, Queens, and the Bronx have more private rooms compared to entire homes/apt

In [ ]:
#Data Analysis 6
#Let's visualize how these room types are spread around NYC

import geopandas as gpd
import matplotlib.pyplot as plt
from geodatasets import get_path

# Load a shapefile of NYC boroughs
nyc = gpd.read_file(get_path('nybb'))
nyc = nyc.to_crs("EPSG:4326")
# Plot the map

#Creating image of NYC
fig, ax = plt.subplots(figsize=(30, 30))
nyc.plot(ax=ax, color='lightblue', edgecolor='black')

#Creting Scatterplot. This works but it does not show the legend correctly.
#colors={'Entire home/apt':'Red', 'Private room':'green', 'Hotel room':'blue', 'Shared room':'yellow'}
#ax2 = plt.scatter(x=listings['longitude'], y=listings['latitude'], c=listings['room_type'].map(colors), label='Map')
#plt.legend(["Entire home/apt", "Private room", 'Hotel room', 'Shared room'],loc="lower left")
#plt.legend()

#This line is optional. You can customize the colors of the dots
color={'Entire home/apt':'Red', 'Private room':'green', 'Hotel room':'blue', 'Shared room':'yellow'}

#Creating Scatterplot. This is the only way I was able to get the legend to appear completely.
for room_type, color in colors.items():
    subset = listings[listings['room_type'] == room_type]
    plt.scatter(subset['longitude'], subset['latitude'], c=color, label=room_type, s=4)

#This line is optional. You can customize the colors of the dots
#colors={'Entire home/apt':'Red', 'Private room':'green', 'Hotel room':'blue', 'Shared room':'yellow'}

#Creating legend. Here we can only change the font of the legend sub titles
legend = plt.legend(loc="lower right", fontsize=10)

#Here we change the font size of the bubbles in legend
for handle in legend.legend_handles:
    handle._sizes = [50]

# Add title and labels
ax.set_title('New York City Boroughs')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')


#You can create the scatterplot using seaborn as well but can't merge with the mat plt that we made first
#ax = sns.lmplot(data=listings, x='longitude',y='latitude', hue='room_type', fit_reg=False)

plt.show()


In [ ]:
#Data Analysis 7
#Let's visualize the price throuhgout NYC

# Load a shapefile of NYC boroughs
nyc = gpd.read_file(get_path('nybb'))
nyc = nyc.to_crs("EPSG:4326")

#Creating image of NYC
fig, ax = plt.subplots(figsize=(30, 30))
nyc.plot(ax=ax, color='lightblue', edgecolor='black')

#Cmap is the color scheme used in the gradient. Alpha is the foggy it will be. S is the size of the bubbles
scatter = plt.scatter(data=listings,x='longitude', y='latitude',cmap='viridis',c='price', alpha=0.7,s=6)
plt.colorbar(scatter, label="price")


plt.show()


Traceback (most recent call last):
  File "c:\Users\marioreyes\.vscode\extensions\ms-python.python-2025.2.0-win32-x64\python_files\python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 16, in <module>
AttributeError: 'str' object has no attribute 'legend_handles'



In [ ]:
#This scatterplot works but it looks like the range of price is being influence by outliers
#Let's see the distribution of the column 'price'

sns.histplot(data=listings, x='price', bins=500, color='blue')
plt.show()

print(listings['price'].describe())
print(listings['price'].quantile([0.1, 0.2, 0.3, 0.4, 0.5,0.6, 0.7, 0.8, 0.9, .95]))
#looks like the 75% of the data is below $250, and 95% if under $550. To better represent this data lets just show 95% of all prices

listings_price = listings[listings['price'] < 550]

sns.histplot(data=listings_price, x='price', bins=100, color='blue')
plt.show()
#That's a better distribution


#Now let's bring over the NYC map
fig, ax = plt.subplots(figsize=(30, 30))
nyc.plot(ax=ax, color='lightblue', edgecolor='black')

#Creating a scatter plot using the df listings_price
scatter = plt.scatter(data=listings_price,x='longitude', y='latitude',c='price', cmap='viridis', alpha=0.7, s=6)
plt.colorbar(scatter, label='Price')

plt.show()

count    22719.000000
mean       216.463885
std        380.588275
min          8.000000
25%         85.000000
50%        149.000000
75%        250.000000
max      20000.000000
Name: price, dtype: float64
0.10     52.0
0.20     74.0
0.30     97.0
0.40    120.0
0.50    149.0
0.60    176.0
0.70    220.0
0.80    294.0
0.90    400.0
0.95    550.0
Name: price, dtype: float64


In [ ]:
#Data Analysis 7 Result
#Looking at 95% of the data (to exclude outliers), the most expensive airBnBs are located in Manhattan.